In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data import generate_dummies

In [3]:
X_train, y_train, qid_train = generate_dummies()
X_valid, y_valid, qid_valid = generate_dummies()

In [5]:
import os
from sklearn.datasets import dump_svmlight_file

os.makedirs("data", exist_ok=True)

dump_svmlight_file(X_train, y_train, "data/train.txt", query_id=qid_train)
dump_svmlight_file(X_valid, y_valid, "data/valid.txt", query_id=qid_valid)

In [9]:
from allrank.data.dataset_loading import load_libsvm_dataset

train, valid = load_libsvm_dataset(
    input_path="data",
    slate_length=240,
    validation_ds_role="valid"
)

In [10]:
from allrank.data.dataset_loading import create_data_loaders

train_loader, valid_loader = create_data_loaders(
    train,
    valid,
    num_workers=1,
    batch_size=64
)

In [11]:
from allrank.models.model_utils import get_torch_device

device = get_torch_device()
print(device)

cpu


In [15]:
from allrank.models.model import FCModel, LTRModel, OutputLayer
from allrank.models.transformer import make_transformer

fc_model_config = {
    "sizes": [64],
    "input_norm": False,
    "activation": None,
    "dropout": 0.0
}
transformer_config = {
    "N": 1,
    "d_ff": 64,
    "h": 1,
    "positional_encoding": None,
    "dropout": 0.0
}

post_model_config = {
    "output_activation": "Sigmoid",
    "d_output": 4
}

fc_model = FCModel(**fc_model_config, n_features=train.shape[0])
transformer = make_transformer(n_features=fc_model.output_size, **transformer_config)
model = LTRModel(fc_model, transformer, OutputLayer(fc_model.output_size, **post_model_config))

model.to(device)

LTRModel(
  (input_layer): FCModel(
    (input_norm): Identity()
    (activation): Identity()
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): Linear(in_features=100, out_features=64, bias=True)
    )
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=64, out_features=64, bias=True)
            (1): Linear(in_features=64, out_features=64, bias=True)
            (2): Linear(in_features=64, out_features=64, bias=True)
            (3): Linear(in_features=64, out_features=64, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=64, out_features=64, bias=True)
          (w_2): Linear(in_features=64, out_features=64, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (sublayer): Mo

In [16]:
import functools

import torch.optim as optim
from allrank.models.losses import ordinal

optimizer = optim.Adam(params=model.parameters(), lr=0.001)
criterion = functools.partial(ordinal, n=4)

In [ ]:
import torch
from allrank.training.train_utils import fit
from allrank.utils.python_utils import dummy_context_mgr

# with dummy_context_mgr():
#     result = fit(
#         model=model,
#         loss_func=criterion,
#         optimizer=optimizer,
#         train_dl=train_loader,
#         valid_dl=valid_loader,
#     )